### Reading the csv file and converting it to a database.


In [3]:
!pip install psycopg2
import pandas as pd

# Load CSV into DataFrame
df = pd.read_csv("../data/freshmart_products.csv")
# checking problems 
print("\n----------------------\n")
print(df.info())  
print("\n----------------------\n")
print(df.isnull().sum()) 
print("\n----------------------\n")
print(df.duplicated().sum())  

print("\n----------------------\n")

# Show first 5 rows
print('first 5 rows on the dataframe')
print("\n----------------------\n")
print(df.head())

# Cleaning Data 
df = df.drop_duplicates()
# handling missing columns 

df["price"] = df["price"].fillna(df["price"].mean())   
df["category"] = df["category"].fillna("unknown")      


# Create a new column StockValue = Price * StockQuantity.
df["StockValue"] = df["price"] * df["stock_quantity"]

print("\n----------------------\n")

print(df["StockValue"].iloc[0:1])

# Aggregate the data by Category to compute:
# Average Price
# Total StockQuantity
agg_df = df.groupby("category").agg(AveragePrice=("price", "mean"),TotalStockQuantity=("stock_quantity", "sum")).reset_index()

# Preview result
print(agg_df)






----------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_iD      50 non-null     int64  
 1   product_name    50 non-null     object 
 2   category        50 non-null     object 
 3   price           50 non-null     float64
 4   stock_quantity  50 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB
None

----------------------

product_iD        0
product_name      0
category          0
price             0
stock_quantity    0
dtype: int64

----------------------

0

----------------------

first 5 rows on the dataframe

----------------------

   product_iD                  product_name             category  price  \
0           1           Honey Garlic Shrimp       Food - Seafood   8.49   
1           2          Vegetable Spiralizer              Kitchen  19.99   
2           3        

### Database Connection and Loading Data 

In [12]:
import psycopg2


db_params = {
    "host": "localhost",
    "database": "FreshMart",
    "user": "newuser",
    "password": "password"
}

def connect_to_db():
    try:
    # Establishing a connection to the database
        connection = psycopg2.connect(**db_params)
        return connection
    except (Exception, psycopg2.Error) as error:
        print(f"Error connecting to the database: {error}")

# Creating tables in the database



try:
    # connect to the database
    conn = connect_to_db()
    cursor = conn.cursor()

    create_table_command = ''' 
        CREATE TABLE products (
            product_id SERIAL PRIMARY KEY,
            product_name VARCHAR(255),
            category VARCHAR(255),
            price DECIMAL,
            stock_quantity INT
        );
    '''
    
    cursor.execute(create_table_command)
    conn.commit()
    print("Table created successfully.")

except (Exception, psycopg2.Error) as error:
    print(f" Error while creating table: {error}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("Database connection closed.")



try:
    # Connect to database
    conn = connect_to_db()
    cursor = conn.cursor()

    # Loop through dataframe rows and insert into DB
    for _, row in df.iterrows():
        cursor.execute(
            """
            INSERT INTO products (product_name, category, price, stock_quantity)
            VALUES (%s, %s, %s, %s)
            """,
            (row["product_name"], row["category"], row["price"], row["stock_quantity"])
        )

    # Commit changes after successful inserts
    conn.commit()
    print("Data inserted successfully into products table!")

except Exception as e:
    # Roll back if something goes wrong
    if conn:
        conn.rollback()
    print(f"Error inserting data: {e}")

finally:
    # Always close connection, whether success or error
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")


Table created successfully.
Database connection closed.
Data inserted successfully into products table!
Database connection closed.


In [13]:
import psycopg2

try:

    conn = connect_to_db()
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM products WHERE category = 'Fitness';")
    dairy_products = cursor.fetchall()  
    print("Products in Dairy category:")
    for row in dairy_products:
        print(row)

    print("\n----------------------\n")

    cursor.execute("SELECT * FROM products WHERE stock_quantity < 50;")
    low_stock_products = cursor.fetchall()
    print("Products with StockQuantity < 50:")
    for row in low_stock_products:
        print(row)

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")

finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")


Products in Dairy category:
(18, 'Fitness Foam Roller', 'Fitness', Decimal('29.99'), 483)
(23, 'Luxury Yoga Mat', 'Fitness', Decimal('39.99'), 386)
(37, 'Protein Powder', 'Fitness', Decimal('44.99'), 463)

----------------------

Products with StockQuantity < 50:
(32, 'Sliced Cucumbers', 'Food - Produce', Decimal('1.29'), 8)
(33, 'Protein Pancake Mix', 'Food - Breakfast', Decimal('3.99'), 47)
(42, 'Peanut Butter Cookies', 'Food - Bakery', Decimal('3.49'), 17)
Database connection closed.
